<a href="https://colab.research.google.com/github/ahmadryan/FC/blob/master/data_binning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import pylab as pl
from numpy import *
%pylab inline
import matplotlib.pyplot as plt

Populating the interactive namespace from numpy and matplotlib


In [11]:
!pip install spacepy
!pip install cdf

     |████████████████████████████████| 860kB 2.7MB/s 
  Created wheel for cdf: filename=CDF-0.32-cp36-cp36m-linux_x86_64.whl size=966388 sha256=2dea6752a03c2ce93c2c743024d1f6b8a8bd077a39835f3c45246f425b0d1367
  Stored in directory: /root/.cache/pip/wheels/56/53/da/dc038be109f8181e4d1a855b6c6b71e41c38d6173325f4640e
Successfully built cdf


In [2]:
!pwd

/content


In [3]:
from google.colab import drive

drive.mount("/content/gdrive", force_remount=True)
!pwd

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
/content


In [4]:
cd /content/gdrive/My\ Drive/Studies/Research/Active_Research/MSC/codes

/content/gdrive/My Drive/Studies/Research/Active_Research/MSC/codes


In [0]:
from numpy import *
from glob import glob
import pandas as pd
import datetime
import time

from scipy.stats import linregress as lrg
#import lmfit

#from scipy.stats import iqr

def fit_func( x0, x, y, scale=None ) :

    if( scale==None ) :

        print( 'Define the scale' )

        return

    elif( scale=='lin' ) :

        xl = x
        yl = y

    elif( scale=='log' ) :

        x = ma.masked_where( x==0, x )
        y = ma.masked_where( y==0, y )
        
        if( min( nanmin( x ), nanmin( y ) ) <= 0 ) :

            raise ValueError( 'Can\'t have values less than or equal to 0' )

            return

        else :

            xl = log10( x )
            yl = log10( y )

    mask = ~isnan( x ) & ~isnan( y )

    fit_param = lrg( xl[mask], yl[mask] )

    y_fit = 10**fit_param[1] * x0**fit_param[0]

    dat = {}

    dat['slope']     = fit_param[0]
    dat['intercept'] = fit_param[1]
    dat['rvalue']    = fit_param[2]
    dat['pvalue']    = fit_param[3]
    dat['sig']       = fit_param[4]

    dat['x_fit']     = x0
    dat['y_fit']     = y_fit

    return dat


## The following function computes the value of each parameter and scales it 
## against the value of the same parameter at 1 AU using an intrapolation 
## technique and using Omni datai and returns a Pandas DataFrame

def trace_func( df=None, key_list=None ) :

    '''The following function computes the value of each parameter and scales it 
    against the value of the same parameter at 1 AU using an intrapolation 
    technique and using Omni datai and returns a Pandas DataFrame
    '''
    import pandas as pd

    from scipy.interpolate import interp1d as spl

    au = 1.496e8 # 1 AU in kms

    df_omni_o = pd.read_pickle( '/content/gdrive/My Drive/Studies/Research/Active_Research/MSC/data/omni/omni_coho1hr_merged_mag_plasma_19630101_20191201_updated.p' )

    df_omni_o['particle_flux'] = ( df_omni_o.np*1e6 ) * ( 
                      df_omni_o.vp_m*1e3) * ( (1.49e11)**2)

    #df_omni = df_omni_o.resample( '4W' ).median( )
    df_omni = df_omni_o.rolling( window='27D', min_periods=100 ).median( )


    t_omni = ( pd.Series( df_omni.index ) - pd.datetime( 1970,1,1,0,0,0,0 )
                                                       ).dt.total_seconds()

    try :

        t_sc_unix = ( ( df.index - pd.datetime(1970,1,1,0,0,0,0) ).total_seconds() )

        print( 'Computing time done' )

    except :

        pass

    # Compute the time at Omni corresponding to time at the spacecraft

    if( isnan( df.vp_r ).all() ) :

        # If all the velocity values in the data is NaN, then we fall back to
        # using the velocity from OMNI data. We compute the minimum and maximum
        # time values for the spacecraft observation, and using that, we find
        # out the average solar wind speed during that time at 1 AU. We use that
        # solar wind speed to find the approximate time. A better method would
        # be to find the approximate time at 1 AU instead of just using the
        # time of spacecraft. Will have to implement that part in a later
        # version

        # TODO: Change the way time is being used to compute the average speed

        t_max = df.index.max()
        t_min = df.index.min()
        dfv = df_omni.vp_r[ ( df_omni.index > t_min ) & ( df_omni.index < t_max ) ]
        vg = dfv.groupby( pd.Grouper( freq='4W' ) ).apply( lambda x: x.median() )

        new_v = zeros_like( df.Tp )
        new_v[:] = nan
        ind1 = 0

        for i in range( len( vg ) ) :

            ind2 = max( where( df.index < vg.index[i] )[0] )

            new_v[ind1:ind2] = vg[i]
            ind1 = ind2

        t_omni_sc = array( t_sc_unix ) - array( ( df.sc_r - 1 )*au/new_v )

    else :

        t_omni_sc = array( t_sc_unix ) - array( ( df.sc_r - 1 )*au/df.vp_r )

    df_intrp = pd.DataFrame( index=df.index )

    df_intrp['sc_r'] = df.sc_r

    for key in key_list :

        intrp_func = spl( t_omni, df_omni[key], fill_value='extrapolate' )

        data_new = intrp_func( t_omni_sc )

        df_intrp[key] = df[key]/data_new

        print( 'Scaling done for %s' %( key ) )

    return df_intrp

In [0]:
#from spacepy.pycdf import CDF as cdf
from numpy import *
from glob import glob
import pandas as pd
import datetime
import pytz
import h5py as hf
import time as tm

#from fitting_func import fit_func, trace_func

start = tm.time()

# Define the function which takes a Pandas DataFrame and file name with number
# of bins to bin the data and save the files

def bin_data( df=None, filename=None, filetype='pickle', n_bin=100, file_version='v100' ) :


    '''Define the function which takes a Pandas DataFrame and file name with number
    of bins to bin the data and save the files

    df = dataframe
    filename = name of the file to which binned data is to be saved
    filetype = [pickle or hdf], default is pickle
    n_bin = number of bins between the smallest and largest distance, 
    default is 100
    file_version = file version number, default is 100
    '''

    dfn= pd.DataFrame()

    for key in key_list :

        #print( 'Binning date for %s ' %( key ) )

        #dfn[key+'_mean'] = empty_like( r_bin )
        #dfn[key+'_mean'] = nan

        dfn[key+'_median'] = empty_like( r_bin )
        dfn[key+'_median'] = nan

        dfn[key+'_iqr_10'] = empty_like( r_bin )
        dfn[key+'_iqr_10'] = nan

        dfn[key+'_iqr_25'] = empty_like( r_bin )
        dfn[key+'_iqr_25'] = nan

        dfn[key+'_iqr_50'] = empty_like( r_bin )
        dfn[key+'_iqr_50'] = nan

        dfn[key+'_iqr_75'] = empty_like( r_bin )
        dfn[key+'_iqr_75'] = nan

        dfn[key+'_iqr_90'] = empty_like( r_bin )
        dfn[key+'_iqr_90'] = nan

        dfn[key+'_num'] = empty_like( r_bin )
        dfn[key+'_num'] = nan


        for ind in range( len( r_bin ) - 1 ) :

            try :

                dat = df[key][(df.sc_r > r_bin[ind]) & 
                              (df.sc_r <= r_bin[ind+1])]

                #dfn[key+'_mean'][ind] = dat.mean()

                dfn[key+'_median'][ind] = dat.median()

                dfn[key+'_iqr_10'][ind] = dat.quantile(0.1)

                dfn[key+'_iqr_25'][ind] = dat.quantile(0.25)

                dfn[key+'_iqr_50'][ind] = dat.quantile(0.50)

                dfn[key+'_iqr_75'][ind] = dat.quantile(0.75)

                dfn[key+'_iqr_90'][ind] = dat.quantile(0.90)

                dfn[key+'_num'][ind] = len(dat)

            except :

                    #print( key+'_mean values set to NaN' )
                    pass

    #print( 'Data binned and saved to a dataframe' )

    # Assign a new name to the pickle file

    #nf = filename[:-2] + '_%s_binned_scaled.p' %( n_bin )

    if( 'pickle' in filetype ) :
        # Save the binned data to a pickel file using protocol 2. Avoid other
        # protocols, since protocol greater than 2 doesn't work for Python 2.*

        fn = filename + '_' + file_version + '.p'
        pd.DataFrame.to_pickle( dfn, fn, protocol=2 )
        print( 'Data written for Pickle file' )

    if( 'hdf' in filetype) :

        fn = filename + '_' + file_version + '.hf'
        hdf=hf.File(fn,'w')
        for i in dfn.columns[0:]: hdf.create_dataset(i,data=np.array(dfn[i]))
        hdf.close()
        print( 'Data written for HDF file' )

    #else :

    
    #raise TypeError( 'File type must be either hdf or pickle' )

    #print( 'Data saved to pickel file for %s' %( nf[54:] ) )

    return( dfn )


In [70]:
!pwd

/content/gdrive/My Drive/Studies/Research/Active_Research/MSC/codes


In [0]:
scaled = True
binning = True

In [104]:
%%time

fnames = sort( glob('/content/gdrive/My Drive/Studies/Research/Active_Research/MSC/data/updated/*updated.p') )

fnames = list( fnames )

print( 'Total %s relevant files found' %( len( fnames ) ) )

# Define the length and size of bins

r_min = -1
r_max = 2
n_bin = 76
r_bin = logspace( r_min, r_max, n_bin )

# Define a custom list of keys/parameters for which you want the binning to be
# done, since it doesn't make much sense to bin the data for parameters like
# time and elevation angle etc

key_list = [ 'br', 'bt', 'bn', 'bm', 'sc_r','np', 'Tp', 'vA', 
             'vp_m','vp_r', 'vp_t', 'vp_n', 'heliographicLatitude', 
    		 'heliographicLongitude', 'zpr', 'zpt', 'zpn', 'zpm', 
             'zmr', 'zmt', 'zmn', 'zmm', 'sig_c', 'parker_angle', 
             'alfven_ratio', 'particle_flux' ]

ldf = []
df_all_l = []

Total 9 relevant files found
CPU times: user 1.34 ms, sys: 0 ns, total: 1.34 ms
Wall time: 3.45 ms


In [105]:
fnames

['/content/gdrive/My Drive/Studies/Research/Active_Research/MSC/data/updated/cassini_coho1hr_merged_mag_plasma_20000101_20040101_updated.p',
 '/content/gdrive/My Drive/Studies/Research/Active_Research/MSC/data/updated/helios1_coho1hr_merged_mag_plasma_19740101_19811201_updated.p',
 '/content/gdrive/My Drive/Studies/Research/Active_Research/MSC/data/updated/helios2_coho1hr_merged_mag_plasma_19760101_19801201_updated.p',
 '/content/gdrive/My Drive/Studies/Research/Active_Research/MSC/data/updated/new_horizons_coho1hr_merged_mag_plasma_20081010210700_updated.p',
 '/content/gdrive/My Drive/Studies/Research/Active_Research/MSC/data/updated/parker_coho1hr_merged_mag_plasma_20180830_20190930_updated.p',
 '/content/gdrive/My Drive/Studies/Research/Active_Research/MSC/data/updated/pioneer10_coho1hr_merged_mag_plasma_19720101_19950901_updated.p',
 '/content/gdrive/My Drive/Studies/Research/Active_Research/MSC/data/updated/pioneer11_coho1hr_merged_mag_plasma_19730101_19941201_updated.p',
 '/conte

In [106]:
%%time

file_version = 'v7'

for f in fnames[0:] :

    df = pd.read_pickle( f )

    #df_all_l.append(df)
    # Adding particle flux to the dataframe (units: 1/second)

    df['particle_flux'] = ( df.np*1e6) * ( df.vp_m*1e3) * ( (df.sc_r*1.49e11)**2)

    print( 'File read and assigned a Dataframe for %s' %( f[36:] ) )

    if( scaled ) :

        # Run the trace function and find the parameters scaled against values at 1 AU

        n_key_list = [ 'Tp', 'bm', 'br', 'bt', 'bn', 'np', 'sig_c', 'vp_m', 'vp_r', 
                     'vp_t', 'vp_n', 'zmm', 'zmr', 'zmt', 'zmn', 'zpm', 'zpr', 'zpt',
                     'zpn', 'vA', 'particle_flux' ]

        if( 'parker' in f ) :

            df.vA = 10**3*df.vA

        dfr = df.copy()

        df_intrp  = trace_func( dfr, key_list=n_key_list )

        df_all_l.append( df_intrp )

        if( binning ) :

            df_temp = bin_data( df_intrp, 
                                filename=f[:-2]+'_%s_binned_scaled' %(n_bin),
                            filetype=['hdf', 'pickle'], n_bin=n_bin, file_version=file_version )

        df = None
        dfr = None
        df_intrp = None
        df_temp = None

    else :

        if( 'parker' in f ) :

            df.vA = 10**3*df.vA

        df_all_l.append( df )

        if( binning ) :

            df_temp = bin_data( df, filename=f[:-2]+'_%s_binned' %(n_bin),
                            filetype=['hdf', 'pickle'], n_bin=n_bin, file_version=file_version )

        df = None
        df_temp = None

File read and assigned a Dataframe for earch/Active_Research/MSC/data/updated/cassini_coho1hr_merged_mag_plasma_20000101_20040101_updated.p
Computing time done
Scaling done for Tp
Scaling done for bm
Scaling done for br
Scaling done for bt
Scaling done for bn
Scaling done for np
Scaling done for sig_c
Scaling done for vp_m
Scaling done for vp_r
Scaling done for vp_t
Scaling done for vp_n
Scaling done for zmm
Scaling done for zmr
Scaling done for zmt
Scaling done for zmn
Scaling done for zpm
Scaling done for zpr
Scaling done for zpt
Scaling done for zpn
Scaling done for vA
Scaling done for particle_flux
Data written for Pickle file
Data written for HDF file
File read and assigned a Dataframe for earch/Active_Research/MSC/data/updated/helios1_coho1hr_merged_mag_plasma_19740101_19811201_updated.p
Computing time done
Scaling done for Tp
Scaling done for bm
Scaling done for br
Scaling done for bt
Scaling done for bn
Scaling done for np
Scaling done for sig_c
Scaling done for vp_m
Scaling do

/usr/local/lib/python3.6/dist-packages/numpy/lib/function_base.py:3942: RuntimeWarning: invalid value encountered in multiply
  x2 = take(ap, indices_above, axis=axis) * weights_above
/usr/local/lib/python3.6/dist-packages/numpy/lib/function_base.py:3955: RuntimeWarning: invalid value encountered in add
  r = add(x1, x2)


Data written for Pickle file
Data written for HDF file
File read and assigned a Dataframe for earch/Active_Research/MSC/data/updated/pioneer10_coho1hr_merged_mag_plasma_19720101_19950901_updated.p
Computing time done
Scaling done for Tp
Scaling done for bm
Scaling done for br
Scaling done for bt
Scaling done for bn
Scaling done for np
Scaling done for sig_c
Scaling done for vp_m
Scaling done for vp_r
Scaling done for vp_t
Scaling done for vp_n
Scaling done for zmm
Scaling done for zmr
Scaling done for zmt
Scaling done for zmn
Scaling done for zpm
Scaling done for zpr
Scaling done for zpt
Scaling done for zpn
Scaling done for vA
Scaling done for particle_flux
Data written for Pickle file
Data written for HDF file
File read and assigned a Dataframe for earch/Active_Research/MSC/data/updated/pioneer11_coho1hr_merged_mag_plasma_19730101_19941201_updated.p
Computing time done
Scaling done for Tp
Scaling done for bm
Scaling done for br
Scaling done for bt
Scaling done for bn
Scaling done for

/usr/local/lib/python3.6/dist-packages/numpy/lib/function_base.py:3942: RuntimeWarning: invalid value encountered in multiply
  x2 = take(ap, indices_above, axis=axis) * weights_above


Data written for Pickle file
Data written for HDF file
File read and assigned a Dataframe for earch/Active_Research/MSC/data/updated/voyager1_coho1hr_merged_mag_plasma_19770101_20181201_updated.p
Computing time done
Scaling done for Tp
Scaling done for bm
Scaling done for br
Scaling done for bt
Scaling done for bn
Scaling done for np
Scaling done for sig_c
Scaling done for vp_m
Scaling done for vp_r
Scaling done for vp_t
Scaling done for vp_n
Scaling done for zmm
Scaling done for zmr
Scaling done for zmt
Scaling done for zmn
Scaling done for zpm
Scaling done for zpr
Scaling done for zpt
Scaling done for zpn
Scaling done for vA
Scaling done for particle_flux
Data written for Pickle file
Data written for HDF file
File read and assigned a Dataframe for earch/Active_Research/MSC/data/updated/voyager2_coho1hr_merged_mag_plasma_19770101_20181201_updated.p
Computing time done
Scaling done for Tp
Scaling done for bm
Scaling done for br
Scaling done for bt
Scaling done for bn
Scaling done for n

/usr/local/lib/python3.6/dist-packages/numpy/lib/function_base.py:3942: RuntimeWarning: invalid value encountered in multiply
  x2 = take(ap, indices_above, axis=axis) * weights_above


Data written for Pickle file
Data written for HDF file
CPU times: user 2min 29s, sys: 1.17 s, total: 2min 30s
Wall time: 2min 33s


In [107]:
%%time

df_all = pd.concat( df_all_l, sort=False)

df_all_l = None

bbm = df_all.br.values**2 + df_all.bt.values**2 + df_all.bn.values**2

sbm = array( [sqrt(xx) for xx in bbm ] )

CPU times: user 2.3 s, sys: 44.9 ms, total: 2.35 s
Wall time: 2.36 s


In [108]:
%%time

df_all.np[   ( df_all.np <=0 ) ] = nan
df_all.vA[   ( df_all.np <=0 ) ] = nan
df_all.Tp[   ( df_all.np <=0 ) ] = nan
df_all.vp_m[ ( df_all.np <=0 ) ] = nan
df_all.vp_r[ ( df_all.np <=0 ) ] = nan
df_all.vp_t[ ( df_all.np <=0 ) ] = nan
df_all.vp_n[ ( df_all.np <=0 ) ] = nan

df_all.particle_flux[ ( df_all.np <=0 ) ] = nan

CPU times: user 237 ms, sys: 9.98 ms, total: 247 ms
Wall time: 250 ms


In [109]:
%%time

df_all['parker_angle'] = arccos(abs(df_all.br)/sbm )

df_all['alfven_ratio'] = 1.e3*df_all.vp_m/df_all.vA

df_all['particle_flux_o'] = df_all.np * df_all.vp_m * df_all.sc_r**2

version = file_version

save_dir = '../data/all_data/'

CPU times: user 86.6 ms, sys: 3.05 ms, total: 89.6 ms
Wall time: 69.5 ms


In [117]:
df_all.keys()

Index(['sc_r', 'Tp', 'bm', 'br', 'bt', 'bn', 'np', 'sig_c', 'vp_m', 'vp_r',
       'vp_t', 'vp_n', 'zmm', 'zmr', 'zmt', 'zmn', 'zpm', 'zpr', 'zpt', 'zpn',
       'vA', 'particle_flux', 'parker_angle', 'alfven_ratio',
       'particle_flux_o'],
      dtype='object')

In [110]:
%%time

if( scaled ) :

    hdf=hf.File(save_dir+'all_spcaecraft_data_scaled_%s.hf' %(version),'w')
    for i in df_all.columns[0:]: hdf.create_dataset(i,data=np.array(df_all[i]))
    hdf.close()

    pd.DataFrame.to_pickle(df_all, save_dir+'all_spcaecraft_data_scaled_%s.p' %(version), protocol=2 )

    dfn = bin_data( df_all, filename=save_dir+'all_spacecraft_data_binned_scaled', 
                                           filetype=['hdf', 'pickle'], n_bin=n_bin, file_version=file_version )

else :

    hdf=hf.File(save_dir+'all_spcaecraft_data_%s.hf' %(version),'w')
    for i in df_all.columns[0:]: hdf.create_dataset(i,data=np.array(df_all[i]))
    hdf.close()

    pd.DataFrame.to_pickle(df_all, save_dir+'all_spcaecraft_data_%s.p' %(version), protocol=2 )

    dfn = bin_data( df_all, filename=save_dir+'all_spacecraft_data_binned',
                                           filetype=['hdf', 'pickle'], n_bin=n_bin, file_version=file_version )

/usr/local/lib/python3.6/dist-packages/numpy/lib/function_base.py:3942: RuntimeWarning: invalid value encountered in multiply
  x2 = take(ap, indices_above, axis=axis) * weights_above


Data written for Pickle file
Data written for HDF file
CPU times: user 24.2 s, sys: 892 ms, total: 25.1 s
Wall time: 27.4 s


In [111]:
!pwd

/content/gdrive/My Drive/Studies/Research/Active_Research/MSC/codes


In [115]:
f = '/content/gdrive/My Drive/Studies/Research/Active_Research/MSC/data/all_data/all_spacecraft_data_binned_v7.hf'
df = hf.File(f)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  


In [116]:
list(df.keys())

['Tp_iqr_10',
 'Tp_iqr_25',
 'Tp_iqr_50',
 'Tp_iqr_75',
 'Tp_iqr_90',
 'Tp_median',
 'Tp_num',
 'alfven_ratio_iqr_10',
 'alfven_ratio_iqr_25',
 'alfven_ratio_iqr_50',
 'alfven_ratio_iqr_75',
 'alfven_ratio_iqr_90',
 'alfven_ratio_median',
 'alfven_ratio_num',
 'bm_iqr_10',
 'bm_iqr_25',
 'bm_iqr_50',
 'bm_iqr_75',
 'bm_iqr_90',
 'bm_median',
 'bm_num',
 'bn_iqr_10',
 'bn_iqr_25',
 'bn_iqr_50',
 'bn_iqr_75',
 'bn_iqr_90',
 'bn_median',
 'bn_num',
 'br_iqr_10',
 'br_iqr_25',
 'br_iqr_50',
 'br_iqr_75',
 'br_iqr_90',
 'br_median',
 'br_num',
 'bt_iqr_10',
 'bt_iqr_25',
 'bt_iqr_50',
 'bt_iqr_75',
 'bt_iqr_90',
 'bt_median',
 'bt_num',
 'heliographicLatitude_iqr_10',
 'heliographicLatitude_iqr_25',
 'heliographicLatitude_iqr_50',
 'heliographicLatitude_iqr_75',
 'heliographicLatitude_iqr_90',
 'heliographicLatitude_median',
 'heliographicLatitude_num',
 'heliographicLongitude_iqr_10',
 'heliographicLongitude_iqr_25',
 'heliographicLongitude_iqr_50',
 'heliographicLongitude_iqr_75',
 'helio

In [0]:
! sudo apt-get install texlive-latex-recommended 
! sudo apt install texlive-latex-extra
! sudo apt install dvipng

In [0]:
# Set the fontstyle to Times New Roman

font = { 'family' : 'Times New Roman', 'weight' : 'normal', 'size' : 10 }
plt.rc( 'font', **font )
plt.rc('text', usetex=True)

In [62]:
# Import and configure the necessary modules.

from numpy import *

import h5py

import matplotlib
import matplotlib.pyplot as plt

import matplotlib.colors
import matplotlib.lines
import matplotlib.patches
import matplotlib.pyplot

matplotlib.rcParams['text.usetex']=False
matplotlib.rcParams['text.latex.unicode']=True

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: MatplotlibDeprecationWarning: 
The text.latex.unicode rcparam was deprecated in Matplotlib 3.0 and will be removed in 3.2.
  from ipykernel import kernelapp as app


In [59]:
!sudo apt-get install texlive-latex-extra texlive-fonts-recommended dvipng

ERROR! Session/line number was not unique in database. History logging moved to new session 61
Reading package lists... Done
Building dependency tree       
Reading state information... Done
dvipng is already the newest version (1.15-1).
texlive-fonts-recommended is already the newest version (2017.20180305-1).
texlive-fonts-recommended set to manually installed.
texlive-latex-extra is already the newest version (2017.20180305-2).
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.


In [67]:
#matplotlib.rcParams['text.latex.preamble'] = '\usepackage{starfont}'


# Define the function for generating a plot.

def plot_composite( key=None, fl=None, scaled=None, ylim=None, yscale=None, ylabel=None, version=1 ) :

    # Define the location of the plot within the figure as well as the size of each.

    inch_plot_x      = 5.00
    inch_plot_y      = 3.00

    inch_mrgn_left   = 0.65
    inch_mrgn_right  = 0.05
    inch_mrgn_bottom = 0.45
    inch_mrgn_top    = 0.20

    inch_fig_x = ( inch_mrgn_left   + inch_plot_x + inch_mrgn_right )
    inch_fig_y = ( inch_mrgn_bottom + inch_plot_y + inch_mrgn_top   )

    figsize = [ inch_fig_x, inch_fig_y ]

    position = [ inch_mrgn_left   / inch_fig_x,
                 inch_mrgn_bottom / inch_fig_y,
                 inch_plot_x      / inch_fig_x,
                 inch_plot_y      / inch_fig_y  ]

    # Define the settings for the plot.

    xlim   = [ 0.1 , 150. ]
    xscale = 'log'
    xlabel = r'\rm Distance from Sun, $r$ [au]'

    if ( scaled ) :
        color = 'm'
    else :
        color = 'b'

    # Initialize the figure.

    fig  = matplotlib.pyplot.figure( figsize=figsize )

    matplotlib.rc( 'text', usetex=True )

    # Initialize the plot.

    plt = fig.add_subplot( 111, position=position )

    matplotlib.pyplot.xlim( xlim )
    matplotlib.pyplot.ylim( ylim )

    matplotlib.pyplot.xscale( xscale )
    matplotlib.pyplot.yscale( yscale )

    matplotlib.pyplot.xlabel( xlabel, size='small' )
    matplotlib.pyplot.ylabel( ylabel, size='small' )

    # Generate the solar-system object markers.

    d_mercury =  0.3871
    d_venus   =  0.7233
    d_earth   =  1.000
    d_mars    =  1.524
    d_ceres   =  2.767
    d_jupiter =  5.204
    d_saturn  =  9.583
    d_uranus  = 19.22
    d_neptune = 30.11
    d_pluto   = 39.48
    d_eris    = 67.781

    lst_d = [ d_mercury, d_venus, d_earth, d_mars, d_ceres,
              d_jupiter, d_saturn, d_uranus, d_neptune, d_pluto, d_eris ]

    #lst_lab = [ r'\starfontserif \Mercury', r'\starfontserif \Venus'  ,
    #            r'\starfontserif \Terra'  , r'\starfontserif \Mars'   ,
    #            r'\starfontserif \Ceres'  , r'\starfontserif \Jupiter',
    #            r'\starfontserif \Saturn' , r'\starfontserif \Uranus' ,
    #            r'\starfontserif \Neptune', r'\starfontserif \Pluto'  ,
    #            r'\starfontserif \Mars'                                 ]

    y = ( ( inch_mrgn_bottom + inch_plot_y ) / inch_fig_y ) + 0.005

    for i in range( len( lst_d ) ) :

        d = lst_d[i]

        if ( ( d == d_ceres ) or ( d == d_pluto ) or ( d == d_eris ) ) :
            c = 'gray'
        else :
            c = 'black'

        if ( d == d_eris ) :
            shift_x  =  -1.3
            shift_y  =  0.005
            rotation = 221
        else :
            shift_x  = 0
            shift_y  = 0
            rotation = 0
        '''
        plt.annotate( lst_lab[i],
                      ( d + shift_x, y + shift_y ),
                      xycoords=( 'data', 'figure fraction' ),
                      color=c, size='medium',
                      horizontalalignment='center',
                      verticalalignment='bottom',
                      rotation=rotation                       )
        '''
        plt.plot( ( d, d ), ylim,
                  color=c, linewidth=0.5, linestyle=':' )

    if ( scaled ) :

        plt.plot( xlim, [ 1., 1. ],
                  color='k', linewidth=0.5, linestyle=':' )

    # Load and plot the data.

    if ( scaled ) :
        dat = h5py.File( '../data/all_data/all_spacecraft_data_binned_scaled_%s.hf' %(version))
    else :
        dat = h5py.File( '../data/all_data/all_spacecraft_data_binned_%s.hf' %(version))

    dat_r  = array( dat['sc_r_iqr_50'] )
    dat_10 = array( dat[key+'_iqr_10'] )
    dat_25 = array( dat[key+'_iqr_25'] )
    dat_50 = array( dat[key+'_iqr_50'] )
    dat_75 = array( dat[key+'_iqr_75'] )

    tk = where( ( isfinite( dat_r  ) ) & ( isfinite( dat_25 ) ) &
                ( isfinite( dat_50 ) ) & ( isfinite( dat_75 ) )   )[0]

    for i in tk :
        plt.plot( [ dat_r[ i], dat_r[ i] ],
                  [ dat_10[i], dat_75[i] ],
                  color=color, linestyle='-', linewidth=0.5 )
        plt.plot( [ dat_r[ i], dat_r[ i] ],
                  [ dat_25[i], dat_75[i] ],
                  color=color, linestyle='-', linewidth=1.0 )

    plt.plot( dat_r[tk], dat_50[tk],
              color=color, linestyle='', marker='o',
              markersize=3.0, markeredgewidth=0      )

    # Output the figure.

    #if ( scaled ) :
    #    fname = 'composite_scaled_' + fl + '_%s.pdf' %(version)
    #else :
    #    fname = 'composite_' + fl + '_%s.pdf' %(version)

    #matplotlib.pyplot.savefig( '../figures/all_data/' + fname )


# Generate the unscaled plots.

version='v6'
plot_composite( key='bm', fl='b', scaled=False,
                ylim=[ 0.01, 200. ], yscale='log',
                ylabel=r'\rm Magnetic Field Strength, $B$ [nT]', version=version )

plot_composite( key='np', fl='np', scaled=False,
                ylim=[ 0.0005, 500. ], yscale='log',
                ylabel=r'\rm Proton Density, $n_p$ [${\rm cm}^{-3}$]', version=version )

plot_composite( key='vp_m', fl='vp', scaled=False,
                ylim=[ 100., 600. ], yscale='linear',
                ylabel=r'\rm Proton Speed, $v_p$ [${\rm km}/{\rm s}$]', version=version )

plot_composite( key='Tp', fl='tp', scaled=False,
                ylim=[ 1.E3, 1.E6 ], yscale='log',
                ylabel=r'\rm Proton Temperature, $T_p$ [K]', version=version )

plot_composite( key='vA', fl='vA', scaled=False,
                ylim=[ 0., 160000. ], yscale='linear',
                ylabel=r"\rm Alf\'en Speed, $v_A$ [${\rm m}/{\rm s}$]", version=version )

plot_composite( key='alfven_ratio', fl='alfven_ratio', scaled=False,
                ylim=[ 1.0, 20.0 ], yscale='linear',
                ylabel=r"$V_p/V_A$", version=version )

plot_composite( key='parker_angle', fl='parker_angle', scaled=False,
                ylim=[ 0.0, 1.8 ], yscale='linear',
                ylabel=r"$cos^{-1}(B_r/|\mathbf{B}|)$ \rm (radians)", version=version )

# Generate the scaled plots.

plot_composite( key='bm', fl='b', scaled=True,
                ylim=[ 0.003, 30. ], yscale='log',
                ylabel=r'\rm Scaled Magnetic Field Strength, $B/\langle B\rangle_{\oplus}$', version=version )

plot_composite( key='np', fl='np', scaled=True,
                ylim=[ 0.00006, 60. ], yscale='log',
                ylabel=r'\rm Scaled Proton Density, $n_p/\langle n_p\rangle_{\oplus}$', version=version )

plot_composite( key='vp_m', fl='vp', scaled=True,
                ylim=[ 0.2, 1.5 ], yscale='linear',
                ylabel=r'\rm Scaled Proton Speed, $v_p/\langle v_p\rangle_{\oplus}$', version=version )

plot_composite( key='Tp', fl='tp', scaled=True,
                ylim=[ 0.02, 20. ], yscale='log',
                ylabel=r'\rm Scaled Proton Temperature, $T_p/\langle T_p\rangle_{\oplus}$', version=version )

plot_composite( key='vA', fl='vA', scaled=True,
                ylim=[ 0.0, 3.2 ], yscale='linear',
                ylabel=r"\rm Scaled Alf\'en Speed, $v_A/\langle v_A\rangle_{\oplus}$", version=version )

plot_composite( key='alfven_ratio', fl='alfven_ratio', scaled=True,
                ylim=[ 1.0, 20.0 ], yscale='linear',
                ylabel=r"$V_p/V_A$", version=version )


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:119: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:119: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:119: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppres

RuntimeError: ignored

<Figure size 410.4x262.8 with 1 Axes>

RuntimeError: ignored

<Figure size 410.4x262.8 with 1 Axes>

...

<Figure size 410.4x262.8 with 1 Axes>

RuntimeError: ignored

<Figure size 410.4x262.8 with 1 Axes>

RuntimeError: ignored

<Figure size 410.4x262.8 with 1 Axes>

In [0]:
plt.show()